In [1]:
!pip install ultralytics
!pip install gdown
!pip install wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.6/615.6 kB 15.7 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=84d168aa144bb161c34e3cbbc538aa7719de00f90aa42c5e4f8ca946616e7368
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:

import wget,gdown
import argparse


def main():
    
    args = "DRIVE LINK OF VIDEO"
    url = args
    if url.split('/')[-1] == '?usp=sharing':
        url= url.replace('?usp=sharing','')
    gdown.download_folder(url)
    print("Folder downloaded")


if __name__=="__main__":
    main()

In [ ]:
from collections import defaultdict

import cv2
import numpy as np

from ultralytics import YOLO
import os
import time
import glob

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


# Load the YOLOv8 model
model = YOLO('WEIGHTS OF THE MODEL (.pt)')

def find_key_and_tuple_by_integer_value(dictionary, target_value):
    for key, value_set in dictionary.items():
        for item in value_set:
            if target_value in item:
                return key, item
    return None, None  # Return None for both key and tuple if the value is not found in any set

# Use the provided video file path
directory_path  = "/kaggle/working/Folder3"
directory_path = str(directory_path)

file_list = glob.glob(os.path.join(directory_path, '*'))
num_files = len(file_list)
print(f"Number of files in the directory: {num_files}")
print("\n")
for file_path in file_list:
    start_time = time.time()

    # Your code to process each file goes here
    print(f"Processing file: {file_path}")


    # Load model names
    class_list = []
    for i in range(len(model.model.names)):
        class_list.append(model.model.names[i])

    area_class_dictionary = {cls: set() for cls in class_list}
    id_manager = []

    # area = [(0, 210), (0, 230), (640, 230), (640, 210)]
    area = [(0, 200), (0, 230), (640, 230), (640, 200)]

    # Open the video file
    video_path = file_path

    cap = cv2.VideoCapture(video_path)

    # Store the track history
    track_history = defaultdict(lambda: [])

    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()

        if success:
            frame = cv2.resize(frame, (640, 640))
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame, persist=True, verbose=False,conf=0.5, iou=0.5)

            # Get the boxes and track IDs
            if results[0].boxes.id is not None:
                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                confidence = results[0].boxes.conf.cpu().tolist()
                class_ = results[0].boxes.cls.int().cpu().tolist()

                # Visualize the results on the frame
                # annotated_frame = results[0].plot()

                # Plot the tracks
                for box, track_id, cnf, cls in zip(boxes, track_ids,confidence,class_):
                    x, y, w, h = box
                    track = track_history[track_id]
                    track.append((float(x), float(y)))  # x, y center point
                    if len(track) > 1:  # retain 90 tracks for 90 frames
                        track.pop(0)
                        cx, cy = track[-1]
                        results = cv2.pointPolygonTest(np.array(area, np.int32), (cx, cy), False)
                        if results >= 0:
                            # cv2.circle(annotated_frame, (int(cx), int(cy)), radius=5, color=(0, 255, 0), thickness=-1)
                            key, value = find_key_and_tuple_by_integer_value(area_class_dictionary, track_id)
                            if key is not None and value is not None and track_id in id_manager:
                                old_confidence = value[1]
                                if cnf>old_confidence and key != model.model.names[cls]:
#                                     print("inside")
                                    area_class_dictionary[key].remove(value)
                                    area_class_dictionary[model.model.names[cls]].add((track_id, cnf))
                            else:
                                area_class_dictionary[model.model.names[cls]].add((track_id, cnf))
                                id_manager.append(track_id)
                # cv2.polylines(annotated_frame, [np.array(area, np.int32)], True, (255, 255, 0), 3)

                vehicle_counts = {
                    vehicle_type: len(ids) for vehicle_type, ids in area_class_dictionary.items()
                }

                # text = "\n".join([f"{vehicle_type} = {count}" for vehicle_type, count in vehicle_counts.items()])
                # text_position = (50, 20)
                # line_height = 20

            #     for line in text.split('\n'):
            #         cv2.putText(annotated_frame, line, text_position, cv2.FONT_HERSHEY_PLAIN, 0.9, (255, 255, 255), 2)
            #         text_position = (text_position[0], text_position[1] + line_height)
            # else:
            #     annotated_frame=results[0].plot()

            # Display the annotated frame
    #         cv2.imshow("YOLOv8 Tracking", annotated_frame)

    #         # Break the loop if 'q' is pressed
    #         if cv2.waitKey(1) & 0xFF == ord("q"):
    #             break
        else:
            # Break the loop if the end of the video is reached
            break

    # # Release the video capture object and close the display window
    cap.release()
    # cv2.destroyAllWindows()
    for _class in class_list:
        count = len(area_class_dictionary.get(_class, []))
        print(f"{_class} = {count}")

    end_time = time.time()

    print('Total time = ', end_time - start_time)

    print("\n\n")
